In [1]:
import pandas as pd
import numpy as np

In [2]:
# ttbar is standard model (SM) event we will treat as background (bkg) we are interested to find beyond SM (BSM) e.g. gluino
# therefore we will treat gluino involved events as bkg
data_bkg = pd.read_csv('/home/debo/UTILITIES/ttbar_out.txt', header=None, skiprows=2)
data_sig = pd.read_csv('/home/debo/UTILITIES/gluino_out.txt', header=None, skiprows=2)

In [3]:
print("data_bkg: ",data_bkg.shape)
print("data_sig: ",data_sig.shape)

('data_bkg: ', (2174419, 95))
('data_sig: ', (1053206, 113))


In [4]:
data_bkg = data_bkg.values
data_sig = data_sig.values

In [5]:
# signal and bkg have different feature size so we will zero pad them to have same feature size
print("data_bkg: ",data_bkg.shape)
print("data_sig: ",data_sig.shape)

('data_bkg: ', (2174419, 95))
('data_sig: ', (1053206, 113))


In [6]:
data_bkg_pad = np.zeros((2174419,18))

In [7]:
data_bkg_padded = np.concatenate((data_bkg, data_bkg_pad),1)


In [8]:
data_bkg_padded.shape

(2174419, 113)

In [9]:
data_bkg_padded[0,:]

array([ 2.63127e+04, -2.24806e+00,  6.34814e+04,  6.89961e-01,
        1.45730e+00,  5.49048e+04, -7.20720e-01, -2.71409e+00,
        2.81850e+04,  1.89810e+00,  2.62792e+00,  0.00000e+00,
        0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,
        0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,
        0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,
        0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,
        0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,
        0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,
        0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,
        0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,
        0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,
        0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,
        0.00000e+00,  8.84342e+04,  2.04707e+00, -5.21237e-01,
        3.96456e+04,  1.99887e+00, -1.93787e+00,  0.00000e+00,
        0.00000e+00,  0.00000e+00,  0.00000e+00,  0.000

In [10]:
# building truth for our dataset signal corresponds to 1 and bkg to 0
truth = np.array(([1] * data_sig.shape[0]) + ([0] * data_bkg_padded.shape[0]))

In [11]:
truth.shape

(3227625,)

In [12]:
truth_r = truth.reshape(3227625,1)
l = np.zeros((3227625,1))
n = l[np.where(truth==0)]
n.size

2174419

In [13]:
total_bk_sig = np.concatenate((data_sig, data_bkg_padded),0)

In [14]:
total_bk_sig.shape

(3227625, 113)

In [15]:
data_truth = np.concatenate((total_bk_sig, truth_r),1)

In [16]:
data_truth.shape

(3227625, 114)

In [17]:
np.random.shuffle(data_truth)

In [18]:
data_truth.shape

(3227625, 114)

In [19]:
data_f = data_truth[:,0:113]
truth_f= data_truth[:,113:114]

In [20]:
data_f.shape

(3227625, 113)

In [118]:
l = np.zeros((3227625, 1))

In [121]:
n = l[np.where(truth_f==0)]

In [21]:
from sklearn.model_selection import cross_val_score, cross_val_predict, train_test_split
from sklearn.model_selection import KFold
import h5py

In [22]:
X_train, X_test, y_train, y_test = train_test_split(data_f, truth_f, train_size=0.75)

In [23]:
print("y_train: ",y_train.shape)
print("y_test: ",y_test.shape)

('y_train: ', (2420718, 1))
('y_test: ', (806907, 1))


In [128]:
l = np.zeros((806907,1))
n = l[np.where(y_test==0)]
n.size

544241

In [129]:
1630178+544241

2174419

In [24]:
Train_ = {
        'X' : X_train,
        'y' : y_train,
    }
Test_ = {
        'X' : X_test,
        'y' : y_test,
    }

In [25]:
import os

In [26]:
SAMPLE_PATH = os.path.join('/home/debo/forgit')

In [27]:
import deepdish.io as io

In [28]:
# Saving train-test data (X:signal+bkg and y:truth)
io.save(os.path.join(SAMPLE_PATH, 'DarkMachine' + 'Train' + '.h5'), Train_)

In [29]:
io.save(os.path.join(SAMPLE_PATH, 'DarkMachine' + 'Test' + '.h5'), Test_)

In [41]:
X_0_train, X_0_test = train_test_split(data_bkg_padded, train_size=0.80)

In [42]:
Train_X0 = {
        'X' : X_0_train,
    }
Test_X0 = {
        'X' : X_0_test,
    }

In [43]:
# Saving bkg data (X: bkg only) to train auto-encoder
io.save(os.path.join(SAMPLE_PATH, 'X_bkg' + 'Train' + '.h5'), Train_X0)

In [44]:
io.save(os.path.join(SAMPLE_PATH, 'X_bkg' + 'Test' + '.h5'), Test_X0)